# Midi to Scores

We want to solve the problem of going from a midi file to music scores in a beautiful and easily readable format.

Using [Magenta](https://magenta.tensorflow.org/) 

# Basic Instructions

1. Double click on the hidden cells to make them visible, or select "View > Expand Sections" in the menu at the top.
2. Hover over the "`[ ]`" in the top-left corner of each cell and click on the "Play" button to run it, in order.
3. Listen to the generated samples.
4. Make it your own: copy the notebook, modify the code, train your own models, upload your own MIDI, etc. See the Magenta code on [GitHub](https://github.com/tensorflow/magenta) for more information!

# Step 0: First things first!
If you're going to use `Magenta`, you need to install it and its dependencies. Some of the later examples will also download other dependencies (such as models and checkpoints)

In [0]:
#@test {"output": "ignore"}

print 'Installing dependencies...'
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

# Temporary hack since the colab installs a RC version of tensorflow.
!pip uninstall -y tensorflow
!pip install -q magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print 'Importing libraries and defining some helper functions...'
from google.colab import files

import magenta.music as mm
import magenta
import tensorflow

print '🎉 Done!'
print magenta.__version__ 
print tensorflow.__version__

# Step 1. Create a NoteSequence

*   List item
*   List item



Everything in `Magenta` is centered around [NoteSequences](https://github.com/tensorflow/magenta/blob/master/magenta/protobuf/music.proto#L27). This is an abstract representation of a series of notes, each with different pitches, instruments and strike velocities, much like [MIDI](https://en.wikipedia.org/wiki/MIDI).


In [0]:
from magenta.protobuf import music_pb2

teapot = music_pb2.NoteSequence()
teapot.notes.add(pitch=69, start_time=0, end_time=0.5, velocity=80)
teapot.notes.add(pitch=71, start_time=0.5, end_time=1, velocity=80)
teapot.notes.add(pitch=73, start_time=1, end_time=1.5, velocity=80)
teapot.notes.add(pitch=74, start_time=1.5, end_time=2, velocity=80)
teapot.notes.add(pitch=76, start_time=2, end_time=2.5, velocity=80)
teapot.notes.add(pitch=81, start_time=3, end_time=4, velocity=80)
teapot.notes.add(pitch=78, start_time=4, end_time=5, velocity=80)
teapot.notes.add(pitch=81, start_time=5, end_time=6, velocity=80)
teapot.notes.add(pitch=76, start_time=6, end_time=8, velocity=80)
teapot.total_time = 8

teapot.tempos.add(qpm=60);

mm.plot_sequence(teapot)
mm.play_sequence(teapot,synth=mm.synthesize)
mm.play_sequence(teapot,synth=mm.fluidsynth)

Loading BokehJS ...

/usr/local/lib/python2.7/dist-packages/pretty_midi/instrument.py:334: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  fade_out = np.linspace(1, 0, .1*fs)


/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/python2.7/dist-packages/fluidsynth.py:394: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return numpy.fromstring(buf[:], dtype=numpy.int16)
/usr/local/lib/pytho

In [0]:
qu_teapot = mm.sequences_lib.quantize_note_sequence(note_sequence=teapot, steps_per_quarter=8)

In [0]:
print (qu_teapot)

time_signatures {
  numerator: 4
  denominator: 4
}
tempos {
  qpm: 60.0
}
notes {
  pitch: 69
  velocity: 80
  end_time: 0.5
  quantized_end_step: 4
}
notes {
  pitch: 71
  velocity: 80
  start_time: 0.5
  end_time: 1.0
  quantized_start_step: 4
  quantized_end_step: 8
}
notes {
  pitch: 73
  velocity: 80
  start_time: 1.0
  end_time: 1.5
  quantized_start_step: 8
  quantized_end_step: 12
}
notes {
  pitch: 74
  velocity: 80
  start_time: 1.5
  end_time: 2.0
  quantized_start_step: 12
  quantized_end_step: 16
}
notes {
  pitch: 76
  velocity: 80
  start_time: 2.0
  end_time: 2.5
  quantized_start_step: 16
  quantized_end_step: 20
}
notes {
  pitch: 81
  velocity: 80
  start_time: 3.0
  end_time: 4.0
  quantized_start_step: 24
  quantized_end_step: 32
}
notes {
  pitch: 78
  velocity: 80
  start_time: 4.0
  end_time: 5.0
  quantized_start_step: 32
  quantized_end_step: 40
}
notes {
  pitch: 81
  velocity: 80
  start_time: 5.0
  end_time: 6.0
  quantized_start_step: 40
  quantized_end_s

In [0]:
mm.plot_sequence(qu_teapot)
mm.play_sequence(qu_teapot,synth=mm.synthesize)

Loading BokehJS ...

/usr/local/lib/python2.7/dist-packages/pretty_midi/instrument.py:334: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  fade_out = np.linspace(1, 0, .1*fs)


In [0]:
teapot = music_pb2.NoteSequence()
teapot.notes.add(pitch=69, start_time=0, end_time=0.4, velocity=80)
teapot.notes.add(pitch=71, start_time=0.4, end_time=1, velocity=80)
teapot.notes.add(pitch=73, start_time=1, end_time=1.5, velocity=80)
teapot.notes.add(pitch=74, start_time=1.5, end_time=2, velocity=80)
teapot.notes.add(pitch=76, start_time=2, end_time=2.5, velocity=80)
teapot.notes.add(pitch=81, start_time=3, end_time=4, velocity=80)
teapot.total_time = 8

teapot.tempos.add(qpm=60);
mm.plot_sequence(teapot)
mm.play_sequence(teapot,synth=mm.synthesize)

Loading BokehJS ...

/usr/local/lib/python2.7/dist-packages/pretty_midi/instrument.py:334: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  fade_out = np.linspace(1, 0, .1*fs)


In [0]:
qu_teapot = mm.sequences_lib.quantize_note_sequence(note_sequence=teapot, steps_per_quarter=2)

In [0]:
print (qu_teapot)

time_signatures {
  numerator: 4
  denominator: 4
}
tempos {
  qpm: 60.0
}
notes {
  pitch: 69
  velocity: 80
  end_time: 0.4
  quantized_end_step: 1
}
notes {
  pitch: 71
  velocity: 80
  start_time: 0.4
  end_time: 1.0
  quantized_start_step: 1
  quantized_end_step: 2
}
notes {
  pitch: 73
  velocity: 80
  start_time: 1.0
  end_time: 1.5
  quantized_start_step: 2
  quantized_end_step: 3
}
notes {
  pitch: 74
  velocity: 80
  start_time: 1.5
  end_time: 2.0
  quantized_start_step: 3
  quantized_end_step: 4
}
notes {
  pitch: 76
  velocity: 80
  start_time: 2.0
  end_time: 2.5
  quantized_start_step: 4
  quantized_end_step: 5
}
notes {
  pitch: 81
  velocity: 80
  start_time: 3.0
  end_time: 4.0
  quantized_start_step: 6
  quantized_end_step: 8
}
total_time: 8.0
quantization_info {
  steps_per_quarter: 2
}
total_quantized_steps: 16



In [0]:
mm.plot_sequence(qu_teapot)
mm.play_sequence(qu_teapot,synth=mm.synthesize)

Loading BokehJS ...

/usr/local/lib/python2.7/dist-packages/pretty_midi/instrument.py:334: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  fade_out = np.linspace(1, 0, .1*fs)


# Better quantization
assuming that we have the tempo information (quarters per minute)

In [0]:
import numpy as np

def find_deltas(seq):
  # Given a NoteSequence, returns a list of deltas between the start times of 
  # the notes
  deltas = []
  prev_start = 0
  for note in seq.notes[1:]:
    start = note.start_time
    delta = start - prev_start
    deltas.append(delta)
    prev_start = start
  return np.array(deltas)

In [0]:
deltas = find_deltas(teapot)

In [0]:
from sklearn.cluster import KMeans  

# Use kmeans as a way to cluster the deltas
# Open question: How to choose number of clusters ahead of time, maybe try
# multiple and compute the variances 
kmeans = KMeans(n_clusters=4, random_state=0)
kmeans.fit(deltas.reshape(-1, 1))
kmeans.labels_
kmeans.cluster_centers_

array([[1. ],
       [0.5],
       [0.6],
       [0.4]])